In [1]:
import math
import string

COMMON_BAD_PASSWORDS = {
    "password", "123456", "123456789", "qwerty",
    "111111", "letmein", "pass123", "admin"
}

def password_strength(password: str) -> dict:
    length = len(password)

    # Character sets
    has_lower = any(c.islower() for c in password)
    has_upper = any(c.isupper() for c in password)
    has_digit = any(c.isdigit() for c in password)
    has_symbol = any(c in string.punctuation for c in password)

    pool_size = 0
    if has_lower:
        pool_size += 26
    if has_upper:
        pool_size += 26
    if has_digit:
        pool_size += 10
    if has_symbol:
        pool_size += len(string.punctuation)

    # Approximate entropy
    entropy = 0
    if pool_size > 0:
        entropy = length * math.log2(pool_size)

    score = 0
    # length points
    if length >= 8:
        score += 1
    if length >= 12:
        score += 1

    # variety points
    score += sum([has_lower, has_upper, has_digit, has_symbol])

    # penalty for common passwords
    is_common = password.lower() in COMMON_BAD_PASSWORDS
    if is_common:
        score = max(0, score - 2)

    # classify
    if score <= 2 or length < 8:
        strength = "weak"
    elif score <= 4:
        strength = "medium"
    else:
        strength = "strong"

    return {
        "password": password,
        "length": length,
        "has_lower": has_lower,
        "has_upper": has_upper,
        "has_digit": has_digit,
        "has_symbol": has_symbol,
        "entropy_bits": round(entropy, 2),
        "score": score,
        "is_common": is_common,
        "strength": strength
    }


# quick test
if __name__ == "__main__":
    for pw in ["Pass123", "MyP@ssw0rd", "correcthorsebatterystaple"]:
        print(password_strength(pw))


{'password': 'Pass123', 'length': 7, 'has_lower': True, 'has_upper': True, 'has_digit': True, 'has_symbol': False, 'entropy_bits': 41.68, 'score': 1, 'is_common': True, 'strength': 'weak'}
{'password': 'MyP@ssw0rd', 'length': 10, 'has_lower': True, 'has_upper': True, 'has_digit': True, 'has_symbol': True, 'entropy_bits': 65.55, 'score': 5, 'is_common': False, 'strength': 'strong'}
{'password': 'correcthorsebatterystaple', 'length': 25, 'has_lower': True, 'has_upper': False, 'has_digit': False, 'has_symbol': False, 'entropy_bits': 117.51, 'score': 3, 'is_common': False, 'strength': 'medium'}


In [2]:
import hashlib
import secrets

PEPPER = "SUPER_SECRET_PEPPER_VALUE"  # Store outside code in real systems

def sha256_no_salt(password: str) -> str:
    return hashlib.sha256(password.encode("utf-8")).hexdigest()

def sha256_with_salt(password: str, salt: str) -> str:
    return hashlib.sha256((salt + password).encode("utf-8")).hexdigest()

def sha256_with_salt_and_pepper(password: str, salt: str, pepper: str = PEPPER) -> str:
    return hashlib.sha256((pepper + salt + password).encode("utf-8")).hexdigest()

# --- Test demo ---
pw = "user123password"

print("=== No salt ===")
h1 = sha256_no_salt(pw)
h2 = sha256_no_salt(pw)
print("hash1:", h1)
print("hash2:", h2)
print("same?", h1 == h2)

print("\n=== With salt ===")
salt1 = secrets.token_hex(16)
salt2 = secrets.token_hex(16)
hs1 = sha256_with_salt(pw, salt1)
hs2 = sha256_with_salt(pw, salt2)
print("salt1:", salt1, "hash1:", hs1)
print("salt2:", salt2, "hash2:", hs2)
print("same?", hs1 == hs2)

print("\n=== With salt + pepper ===")
hp1 = sha256_with_salt_and_pepper(pw, salt1)
print("hash (salt+pepper):", hp1)


=== No salt ===
hash1: 6384e02d9a47ac61c64950915c878ea54cc91a3d9c11d7497818017db8356d2a
hash2: 6384e02d9a47ac61c64950915c878ea54cc91a3d9c11d7497818017db8356d2a
same? True

=== With salt ===
salt1: 93629f0907604114d1924c99823e0835 hash1: 85b004e81613a47815370f310a58e5e67a4f2c9a57412f91ca1e087262ea80cd
salt2: b42a2cab678b9cae8d17fa3c0a3226d0 hash2: 50a790143bdf8682e1f376952f61e718a997a4ba137bbfba71b905022a4385f3
same? False

=== With salt + pepper ===
hash (salt+pepper): 2b8c374585a3f5ffa85457473590f52b43f47cfc3118b992132b5327e7541a95


In [3]:
import hashlib
import secrets

PEPPER = "SUPER_SECRET_PEPPER_VALUE"  # in real life: env var, not in code!

def sha256_no_salt(password: str) -> str:
    return hashlib.sha256(password.encode("utf-8")).hexdigest()

def sha256_with_salt(password: str, salt: str) -> str:
    return hashlib.sha256((salt + password).encode("utf-8")).hexdigest()

def sha256_with_salt_and_pepper(password: str, salt: str, pepper: str = PEPPER) -> str:
    return hashlib.sha256((pepper + salt + password).encode("utf-8")).hexdigest()

if __name__ == "__main__":
    pw = "user123password"

    print("=== No salt ===")
    h1 = sha256_no_salt(pw)
    h2 = sha256_no_salt(pw)
    print("hash1:", h1)
    print("hash2:", h2)
    print("same?", h1 == h2)

    print("\n=== With salt ===")
    salt1 = secrets.token_hex(16)
    salt2 = secrets.token_hex(16)
    hs1 = sha256_with_salt(pw, salt1)
    hs2 = sha256_with_salt(pw, salt2)
    print("salt1:", salt1, "hash1:", hs1)
    print("salt2:", salt2, "hash2:", hs2)
    print("same?", hs1 == hs2)

    print("\n=== With salt + pepper ===")
    hp1 = sha256_with_salt_and_pepper(pw, salt1)
    print("hash (salt+pepper):", hp1)


=== No salt ===
hash1: 6384e02d9a47ac61c64950915c878ea54cc91a3d9c11d7497818017db8356d2a
hash2: 6384e02d9a47ac61c64950915c878ea54cc91a3d9c11d7497818017db8356d2a
same? True

=== With salt ===
salt1: b50dd50a9a4885a584abd57f0d1cc877 hash1: 1a40bd3b6bf3a54dcb651595dd2a14c43097b5bc65aa4baff9060f5359c5fc65
salt2: 09e622506754a922eccc529c055e1677 hash2: b8e9675d2a6e4e8e53a12a7314df4315ae0abff1ed718136b48ad89b5ea69926
same? False

=== With salt + pepper ===
hash (salt+pepper): a9550282d17970b09cb72bbdcd0dc5716c738c2de1234670cebcce880724d809


In [4]:
import pyotp
import qrcode

def create_totp_secret() -> str:
    # Base32 secret for TOTP
    return pyotp.random_base32()

def generate_totp_uri(secret: str, username: str, issuer: str = "MySecureApp") -> str:
    totp = pyotp.TOTP(secret)
    uri = totp.provisioning_uri(name=username, issuer_name=issuer)
    return uri

def create_qr_code(uri: str, filename: str = "totp_qr.png"):
    img = qrcode.make(uri)
    img.save(filename)
    print(f"QR code saved to {filename}")

def current_totp_code(secret: str) -> str:
    totp = pyotp.TOTP(secret)
    return totp.now()

def verify_totp_code(secret: str, code: str, window: int = 1) -> bool:
    totp = pyotp.TOTP(secret)
    return totp.verify(code, valid_window=window)


if __name__ == "__main__":
    user = "alice@example.com"
    secret = create_totp_secret()
    uri = generate_totp_uri(secret, user)
    create_qr_code(uri)

    print("Secret:", secret)
    print("Scan totp_qr.png with an authenticator app.")
    print("Current TOTP code:", current_totp_code(secret))


QR code saved to totp_qr.png
Secret: UI3CKJW6HW2RMRUBTDUSGYR2QQMDXALV
Scan totp_qr.png with an authenticator app.
Current TOTP code: 072458


In [5]:
import time
import hashlib

# --- Hash functions ---
def hash_md5(password: str) -> str:
    return hashlib.md5(password.encode("utf-8")).hexdigest()

def hash_sha256(password: str) -> str:
    return hashlib.sha256(password.encode("utf-8")).hexdigest()

# --- Common password dictionary ---
COMMON_PASSWORDS = [
    "123456", "password", "123456789", "12345678",
    "qwerty", "abc123", "password1", "iloveyou"
]

# --- Brute-force simulation ---
def simulate_bruteforce_sha(password: str, hash_type: str = "md5"):
    if hash_type == "md5":
        target = hash_md5(password)
        hash_func = hash_md5
    elif hash_type == "sha256":
        target = hash_sha256(password)
        hash_func = hash_sha256
    else:
        raise ValueError("hash_type must be 'md5' or 'sha256'")

    print(f"Target hash ({hash_type}) for '{password}': {target}")
    print("Starting dictionary attack...\n")

    start = time.time()
    for guess in COMMON_PASSWORDS:
        h = hash_func(guess)
        print(f"Trying '{guess}' -> {h}")
        if h == target:
            elapsed = time.time() - start
            print(f"\nSUCCESS! Found password '{guess}' in {elapsed:.6f} seconds.")
            return

    elapsed = time.time() - start
    print(f"\nFAILED. Password not in dictionary. Time: {elapsed:.6f} seconds.")

# --- Run demo ---
if __name__ == "__main__":
    simulate_bruteforce_sha("password", "md5")
    simulate_bruteforce_sha("password", "sha256")
    print("\nbcrypt is intentionally slow; we normally *wouldn't* brute-force it in a demo.")


Target hash (md5) for 'password': 5f4dcc3b5aa765d61d8327deb882cf99
Starting dictionary attack...

Trying '123456' -> e10adc3949ba59abbe56e057f20f883e
Trying 'password' -> 5f4dcc3b5aa765d61d8327deb882cf99

SUCCESS! Found password 'password' in 0.000016 seconds.
Target hash (sha256) for 'password': 5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8
Starting dictionary attack...

Trying '123456' -> 8d969eef6ecad3c29a3a629280e686cf0c3f5d5a86aff3ca12020c923adc6c92
Trying 'password' -> 5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8

SUCCESS! Found password 'password' in 0.000011 seconds.

bcrypt is intentionally slow; we normally *wouldn't* brute-force it in a demo.


In [6]:
import bcrypt
import pyotp

# bcrypt hashing
def hash_bcrypt(password: str) -> bytes:
    return bcrypt.hashpw(password.encode(), bcrypt.gensalt())

def verify_bcrypt(password: str, hashed: bytes) -> bool:
    return bcrypt.checkpw(password.encode(), hashed)

# TOTP (2FA)
def create_totp_secret() -> str:
    return pyotp.random_base32()

def current_totp_code(secret: str) -> str:
    return pyotp.TOTP(secret).now()

def verify_totp_code(secret: str, code: str) -> bool:
    return pyotp.TOTP(secret).verify(code)


In [7]:
class User:
    def __init__(self, username: str, password_hash: bytes, totp_secret: str):
        self.username = username
        self.password_hash = password_hash
        self.totp_secret = totp_secret


class AuthSystem:
    def __init__(self):
        self.users = {}

    def register_user(self, username: str, password: str) -> bool:
        if username in self.users:
            print("Username already exists.")
            return False

        info = password_strength(password)
        if info["strength"] == "weak" or info["is_common"]:
            print("Password too weak or common. Please choose a stronger one.")
            print(info)
            return False

        pw_hash = hash_bcrypt(password)
        totp_secret = create_totp_secret()

        self.users[username] = User(username, pw_hash, totp_secret)
        print(f"User '{username}' registered successfully.")
        print(f"TOTP secret (store securely, or encode as QR): {totp_secret}")
        return True

    def authenticate(self, username: str, password: str, totp_code: str = None) -> bool:
        user = self.users.get(username)
        if not user:
            print("User not found.")
            return False

        if not verify_bcrypt(password, user.password_hash):
            print("Invalid password.")
            return False

        if totp_code is not None:
            if not verify_totp_code(user.totp_secret, totp_code):
                print("Invalid TOTP code.")
                return False
        else:
            print("Warning: no TOTP code provided (2FA not verified).")

        print("Authentication successful!")
        return True


if __name__ == "__main__":
    auth = AuthSystem()

    auth.register_user("alice", "MyP@ssw0rd123!")

    alice_secret = auth.users["alice"].totp_secret
    alice_code = current_totp_code(alice_secret)
    print("Example TOTP code for Alice:", alice_code)

    auth.authenticate("alice", "MyP@ssw0rd123!", alice_code)

    auth.authenticate("alice", "wrongpass", alice_code)

    auth.authenticate("alice", "MyP@ssw0rd123!", "000000")


User 'alice' registered successfully.
TOTP secret (store securely, or encode as QR): PZYKXJD6CI5L5GJDKVSA72NSZGKKMAYD
Example TOTP code for Alice: 288984
Authentication successful!
Invalid password.
Invalid TOTP code.
